In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)

In [2]:
balance = 10000
tickers = ['SPY', 'VOX', 'VCR', 'VDC', 'VDE', 'VFH', 'VHT', 'VIS', 'VGT', 'VAW', 'VNQ', 'VPU']
etfs = {}
for ticker in tickers:
    stock = yf.Ticker(ticker)
    etfs[ticker] = {}
    etfs[ticker]['info'] = stock.info
    h = stock.history(period='10y', interval='1mo')
    h['Change'] = h['Close'] - h['Open']
    h['%Change'] = h['Change'] / h['Open'] * 100
    etfs[ticker]['history'] = h

In [10]:
data = pd.DataFrame()
for ticker in tickers:
    df = etfs[ticker]['history']
    df = df[df['%Change'].notnull()]
    data[ticker] = df['%Change']

data

,SPY,VOX,VCR,VDC,VDE,VFH,VHT,VIS,VGT,VAW,VNQ,VPU
Date,,,,,,,,,,,,
2010-05-01,-8.383725,-4.794683,-7.709520,-5.041860,-12.367388,-9.562044,-7.199832,-9.408974,-8.200234,-9.544837,-6.136878,-6.009144
2010-06-01,-4.742547,-1.478326,-8.804469,-1.623120,-4.535528,-5.791191,-0.482870,-5.977062,-5.544851,-6.884127,-4.825901,-1.257862
2010-07-01,6.894109,9.211196,8.118715,6.447713,7.891037,7.130510,0.830641,9.974184,7.110507,12.046474,9.498094,7.559092
2010-08-01,-5.969338,-2.007383,-6.189944,-2.682927,-6.399617,-8.942766,-3.276836,-8.416156,-8.061821,-4.055266,-2.854696,-0.386183
2010-09-01,6.925271,7.988853,10.930889,4.605138,8.470588,4.780362,8.300669,10.195721,11.111111,7.091413,2.352599,0.956633
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-01,-0.559614,0.053056,0.477891,-0.789016,-12.289395,-2.773761,-2.915604,-1.175706,2.885006,-6.965471,0.789445,5.905512
2020-02-01,-8.377403,-6.385441,-8.073404,-8.413957,-14.847285,-11.672942,-6.816607,-9.532857,-7.389435,-9.417398,-6.977992,-10.216513
2020-03-01,-13.569670,-15.271430,-18.234100,-6.778510,-38.399869,-23.477601,-5.633723,-20.995032,-11.482010,-16.232062,-20.305853,-10.675317


In [12]:
data.idxmin(axis=1)

Date
2010-05-01    VDE
2010-06-01    VCR
2010-07-01    VHT
2010-08-01    VFH
2010-09-01    VPU
             ... 
2020-01-01    VDE
2020-02-01    VDE
2020-03-01    VDE
2020-04-01    VIS
2020-04-23    VPU
Length: 121, dtype: object

In [4]:
data = yf.download(
    tickers=" ".join(tickers),
    period="10y",
    interval="1mo",
    group_by="ticker",
    auto_adjust=True,
    threads=True
)

[*********************100%***********************]  12 of 12 completed


In [19]:
df_change = pd.DataFrame()
for ticker in tickers:
    #data.loc[:, (ticker, 'Change')] = data[ticker]['Close'] - data[ticker]['Open']
    data.loc[:, (ticker, 'PctChange')] = \
        ((data[ticker]['Close'] - data[ticker]['Open']) / data[ticker]['Open']) * 100
    df_change[ticker] = data[ticker]['PctChange']
    
df_change = df_change.dropna(how='all')
df_change


,SPY,VOX,VCR,VDC,VDE,VFH,VHT,VIS,VGT,VAW,VNQ,VPU
Date,,,,,,,,,,,,
2010-05-01,-8.384985,-4.783681,-7.719799,-5.049198,-12.369610,-9.560803,-7.195804,-9.423654,-8.201559,-9.542986,-6.159970,-5.994759
2010-06-01,-4.734654,-1.466488,-8.814287,-1.631524,-4.547243,-5.776299,-0.495148,-5.987806,-5.538582,-6.871413,-4.811627,-1.252882
2010-07-01,6.902564,9.198871,8.123250,6.439988,7.897987,7.135251,0.814785,9.970501,7.111112,12.035335,9.522786,7.562323
2010-08-01,-5.964819,-1.999997,-6.203376,-2.684570,-6.399402,-8.915746,-3.270392,-8.416017,-8.049189,-4.050487,-2.858246,-0.383436
2010-09-01,6.933377,7.983482,10.945897,4.587154,8.469919,4.775382,8.301888,10.202140,11.111106,7.083526,2.338382,0.957443
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-01,-0.559435,0.052936,0.476808,-0.785748,-12.289479,-2.768712,-2.918425,-1.175943,2.886051,-6.966386,0.794074,5.910302
2020-02-01,-8.377917,-6.391684,-8.070957,-8.416216,-14.848864,-11.676684,-6.815165,-9.534675,-7.388486,-9.416436,-6.976987,-10.217776
2020-03-01,-13.567618,-15.280247,-18.232442,-6.777590,-38.404510,-23.476158,-5.630684,-20.998357,-11.479660,-16.234327,-20.308046,-10.673992
